<a href="https://colab.research.google.com/github/cienacamille/cecs457/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Deep Neural Networks Project

In this project, you will be working with a real-world data set from the Las Vegas Metropolitan Police Department. The dataset  contains information about the reported incidents, including the time and location of the crime, type of incident, and number of persons involved.

The dataset is downloaded from the public docket at:
https://opendata-lvmpd.hub.arcgis.com

let's read the csv file and transform the data:

In [2]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset, TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError


In [3]:
orig_df = pd.read_csv('../content/LVMPD-Stats.csv', parse_dates=['ReportedOn'])

In [4]:
df = pd.read_csv('../content/LVMPD-Stats.csv', parse_dates=['ReportedOn'],
                 usecols = ['X', 'Y', 'ReportedOn',
                            'Area_Command','NIBRSOffenseCode',
                            'VictimCount' ] )

df['DayOfWeek'] = df['ReportedOn'].dt.day_name()
df['Time' ]     = df['ReportedOn'].dt.hour
df.drop(columns = 'ReportedOn', inplace=True)

In [5]:

df['X'] = df['X']
df['Y'] = df['Y']
df['Time'] = pd.factorize(df['Time'])[0]
df['DayOfWeek'] = pd.factorize(df['DayOfWeek'])[0]
df.Area_Command = pd.factorize(df['Area_Command'])[0]
df.VictimCount = pd.factorize(df['VictimCount'])[0]
df.NIBRSOffenseCode = pd.factorize(df['NIBRSOffenseCode'])[0]
df.dropna(inplace=True)

In [6]:
df= df[['X', 'Y', 'Area_Command', 'NIBRSOffenseCode',
       'DayOfWeek', 'Time','VictimCount']]

In [7]:
df.values.shape

(275, 7)

# Goal
The goal is to build a predictive model that is trained on the following data:
* latitude and longitude (location)
* Hour of the day
* Day of the week
* Area-of-command code: The police designation of the bureau of the operation.
* Classification code for the crime committed
  
The predicted variable is the number of persons involved in the accident.


## Task 1
* print a few rows of the values in the dataframe ``df`` and explain what each column of data means.
* identify the input and target variables
* what is the range of values in each column? Do you need to scale, shift or normalize your data?


In [8]:
df.head()

,X,Y,Area_Command,NIBRSOffenseCode,DayOfWeek,Time,VictimCount
0,-115.087518,36.216702,0,0,0,0,0
1,-115.240172,36.189693,1,1,1,1,0
2,-115.143088,36.181329,2,1,2,0,1
3,-115.225014,36.117633,3,1,1,2,2
4,-115.176708,36.095967,4,1,1,3,0


target variable: VictimCount

input variable: all other variables
* X
  * longitude
  * range: [-115, -116]
* Y
  * latitude
  * range: [36, 37]
* Area_Command
  * acronym of the area command dealing with the crime
  * range: [0, number of area commands in Las Vegas]
* NIBRSOffenseCode
  * code pertaining to the type of offense that the crime falls under
  * range: [0, number of NIBRS Offense Codes]
* DayOfWeek
  * day of the week that the crime was reported
  * range: [0, 6]
* Time
  * time that the crime was reported
  * range: [0, 23]
* VictimCount
  * number of victims involved in the crime
  * range: [0, population of Las Vegas]

## Task 2

* Create two `DataLoader` objects for training and testing based on the input and output variables. Pick a reasonable batch size and verify the shape of data by iterating over the one dataset and printing the shape of the batched data.

In [9]:
my_x = np.array(df.iloc[:, 0:6]) # a list of numpy arrays
my_y = np.array(df.iloc[:, 6:]) # another list of numpy arrays (targets)

tensor_x = torch.Tensor(my_x) # transform to torch tensor
tensor_y = torch.Tensor(my_y)

X_train, X_test, y_train, y_test = train_test_split(tensor_x, tensor_y, train_size=0.7)

train_dataset = TensorDataset(X_train,y_train) # create your dataset
train_dataloader = DataLoader(train_dataset, batch_size= 16) # create your dataloader

test_dataset = TensorDataset(X_test,y_test) # create your dataset
test_dataloader = DataLoader(test_dataset, batch_size= 10) # create your dataloader

print(X_train.shape)
print(X_test.shape)


torch.Size([192, 6])
torch.Size([83, 6])


## Task 3
In this task you will try to predict number of crime victims as a **real number**. Therefore the machine learning problem is a **regression** problem.

* Define the proper loss function for this task
* what should the size of the predicted output be?
* explain your choice of architecture, including how many layers you will be using
* define an optimizer for training this model, choose a proper learning rate
* write a training loop that obtains a batch out of the  training data and calculates the forward and backward passes over the neural network. Call the optimizer to update the weights of the neural network.
* write a for loop that continues the training over a number of epochs. At the end of each epoch, calculate the ``MSE`` error on the test data and print it.
* is your model training well? Adjust the learning rate, hidden size of the network, and try different activation functions and number of layers to achieve the best accuracy and report it.

In [10]:
layer_1 = Dense(units=6, activation='relu')
a1 = layer_1(my_x)

layer_2 = Dense(units=1, activation='relu')
a2 = layer_2(a1)

model = Sequential([layer_1, layer_2])

model.compile(optimizer='sgd',
              loss = 'mean_squared_error')

model.fit(my_x, my_y, epochs=100)



Epoch 1/100
9/9 [==============================] - 0s 3ms/step - loss: 1.4618
Epoch 2/100
9/9 [==============================] - 0s 37ms/step - loss: 1.4618
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 1.4618
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 1.4618
Epoch 5/100
9/9 [==============================] - 0s 4ms/step - loss: 1.4618
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 1.4618
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 1.4618
Epoch 8/100
9/9 [==============================] - 0s 3ms/step - loss: 1.4618
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 1.4618
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 1.4618
Epoch 11/100
9/9 [==============================] - 0s 2ms/step - loss: 1.4618
Epoch 12/100
9/9 [==============================] - 0s 3ms/step - loss: 1.4618
Epoch 13/100
9/9 [==============================] - 0s 2ms/s

In [11]:
y_mse = model.predict([[-115.303952, 36.019455, 3, 1, 2, 1]])
y_mse

1/1 [==============================] - 0s 112ms/step


array([[0.]], dtype=float32)

## Task 4

In this task, you will try to predict the number of crime victims as a **class number**. Therefore the machine learning problem is a **classification** problem.

* Repeat all the steps in task 3. Specifically, pay attention to the differences with regression.
* How would you find the number of classes on the output data?
* How is the architecture different?
* How is the loss function different?
* Calculate the Accuracy for test data as the number of correct classified outputs divided by the total number of test data in each epoch. Report it at the end of each epoch
* Try a few variations of learning rate, hidden dimensions, layers, etc. What is the best accuracy that you can get?

In [12]:
layer_1 = Dense(units=6, activation='sigmoid')
a1 = layer_1(my_x)

layer_2 = Dense(units=1, activation='sigmoid')
a2 = layer_2(a1)

model = Sequential([layer_1, layer_2])

model.compile(optimizer='sgd',
              loss = 'mean_squared_error')

model.fit(my_x, my_y, epochs=100)


Epoch 1/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9578
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9549
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9543
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9541
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9541
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9541
Epoch 7/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9540
Epoch 8/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9540
Epoch 9/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9540
Epoch 10/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9540
Epoch 11/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9540
Epoch 12/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9540
Epoch 13/100
9/9 [==============================] - 0s 2ms/st

In [13]:
y_mse = model.predict([[-115.303952, 36.019455, 3, 1, 2, 1]])
y_mse

1/1 [==============================] - 0s 114ms/step


array([[0.71232617]], dtype=float32)

## Task 5

### Reflect on your results

* Write a paragraph about your experience with tasks 3 and 4. How do you compare the results? Which one worked better? Why?
* Write a piece of code that finds an example of a  miss-classification. Calculate the probabilities for the output classes and plot them in a bar chart. Also, indicate what is the correct class label.

## Task 6: Exploring the patterns in raw data

* Plot the crime incidents as a `scatter` plot using the corrdinates. Use the color property of each datapoint to indicate the day of the week. Is there a pattern in the plot?
* Now make a new scatter plot and use the color property of each datapoint to indicate the number of persons involved in the incident. Is there a pattern here?
* use numpy (or pandas if you like) to sort the number of crimes reported by the day of the week. What days are most frequent?
